In [ ]:
import numpy as np
from numpy.random import rand
import numba
import time
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
L = 4 # length of the lattice
dim = 2 # dimension of the lattice
cold = True # cold start or hot start

T_min, T_max = 0.1, 5.1
dT = 0.1
nt = 1 + np.int64(np.round(np.round((T_max-T_min) / dT)))
T = np.linspace(T_min, T_max, nt)

eqSteps = 10**5 # mixing time
mcSteps = 10**6 # MC steps = number of samples
sampling_interval = 10 # sampling interval

N = L**dim # number of spins

In [ ]:
save_folder = f"./data/L={L}_Tmin={np.round(T_min,1)}_Tmax={np.round(T_max,1)}_eqSteps={eqSteps}_mcSteps={mcSteps}_interval={sampling_interval}/state/raw"
os.makedirs(save_folder, exist_ok=True)

In [ ]:
def ArrayToString(arr):
    int_arr = arr.astype(int) # Convert the array to an integer array
    str_arr = ''.join(str(i) for i in int_arr) # Convert the integer array to a string
    return str_arr

def SpinToBinary(IsingData):
    return (IsingData + 1) // 2

def BinaryToSpin(BernoulliData):
    return BernoulliData * 2 - 1

def initialstate(L): # -1 = spin down, 1 = spin up
    state = -np.ones((L,L)) # all spin down
    return state

@numba.jit(nopython=True)
def mcmove(config, beta):
    range_i = np.arange(0, L)
    range_j = np.arange(0, L)
    np.random.shuffle(range_i)
    np.random.shuffle(range_j)
    for i in range_i:
        for j in range_j:
            s = config[i, j]
            nb = config[(i+1) % L, j] + config[i, (j+1) % L] + config[(i-1) % L, j] + config[i, (j-1) % L]
            cost = 2*s*nb
            if cost < 0:
                s *= -1
            elif rand() < np.exp(-cost*beta):
                s *= -1
            config[i, j] = s
    return config
    
@numba.jit(nopython=True)
def calcEne(config):
    energy = 0
    for i in range(L):
        for j in range(L):
            S = config[i, j]
            nb = config[(i+1) % L, j] + config[i, (j+1) % L] + config[(i-1) % L, j] + config[i, (j-1) % L]
            energy += -nb*S
    return energy/2.  # To compensate for over-counting

@numba.jit(nopython=True)
def calcMag(config):
    # Magnetization of a given configuration
    mag = np.sum(config)/N
    return abs(mag)

@numba.jit(nopython=True)
def z2(config):
    # Z2 symmetry of a given configuration
    z2_mul = np.random.randint(0,2,(config.shape[0],1)) * 2 - 1
    config *= z2_mul

In [ ]:
# ----------------------------main program------------------------------
T_crit = 2/np.log(1+np.sqrt(2)) # critical temperature
T = np.round(T, 2) 

def mcmc(idx=0):
    os.makedirs(f"{save_folder}/run={idx}", exist_ok=True)
    start = time.perf_counter()
  
    spin_config = initialstate(L) 
    configs_flatten = np.zeros((mcSteps, N)) 
    for tt in tqdm(range(nt)):
        iT = 1.0/T[tt]
        iT2 = iT*iT
        for i in range(eqSteps):        
            mcmove(spin_config, iT)
        for i in range(mcSteps):        
            for j in range(sampling_interval):
                mcmove(spin_config, iT)
            config_flatten = spin_config.flatten()
            configs_flatten[i, :] = config_flatten
        z2(configs_flatten) # apply z2 symmetry
        np.save(f"{save_folder}/run={idx}/T={T[tt]}.npy",configs_flatten.astype(np.short))
        
    end = time.perf_counter()
    elapsed = end - start
    np.save(f"{save_folder}/run={idx}/time.npy", np.array([elapsed]))
    return elapsed

In [ ]:
n = 5
for idx in range(n):
    elapsed = mcmc(idx)
    print(f"{idx} run elapsed: {elapsed} sec")